# Why Some Beers Spark Debate: A Short Walk Through Controversial Brews

## Overview

### 1. Understanding and treating the data (finished)
- **1.1 Data conversion**
- **1.2 Data exploration**
- **1.3 Dataset Merging**
### 2. Exploring the definition of controversiality (finised, modify two values)
- **2.1 Ratings and reviews filtering** (finised, need threshold)
- **2.2 Naive classification** (finised)
- **2.3 Clustering the beers** (finished, need elbow to choose)
### 3. Extracting information from the textual reviews (just validate pls)
- **3.1 Sentiment analyses** (finished, but Gustave check pls)
- **3.2 Comparing the distribution with the other attributes** (finished)
- **3.3 Clustering and comparison against the previous clusters** (finished)
- **3.4 Understanding the difference between the two clusters** (finished)
### 4. Analyzing potential reasons of controversiality (in construction)
- **4.1 Effect of the alcohol by volume** (in construction)
- **4.2 Effect of the number of ratings** (finished)
- **4.3 Effect of the expertise level of the user** (in construction)
### 5. Countries influence on the controversiality (in construction)
- **5.1 Importance of the origin of the user** (in construction)
- **5.2 Importance of the origin of the beer** (in construction)
- **5.3 Relation between origin of the user and origin of the beer** (finished)
### 6. Future work (in construction)
### 7. Conclusion (in construction)

TO DO : DONT FORGET SOMEHWERE TO QUICKLY OCMPARE THE DIFFERENT IN VARIANCE OF THE TWO DATASET (then explain the loss of resolution leading to maybe higher variance)

Reload necessary functions

In [44]:
from importlib import reload
import src.utils.utils_functions
reload(src.utils.utils_functions)
from src.utils.utils_functions import *

import src.utils.clustering_GMM
reload(src.utils.clustering_GMM)
from src.utils.clustering_GMM import *

import src.utils.aabv_analysis
reload(src.utils.aabv_analysis)
from src.utils.aabv_analysis import *

import src.utils.nbr_rating_analysis
reload(src.utils.nbr_rating_analysis)
from src.utils.nbr_rating_analysis import *

import src.utils.user_analysis
reload(src.utils.user_analysis)
from src.utils.user_analysis import *

import src.utils.origin_of_user
reload(src.utils.origin_of_user)
from src.utils.origin_of_user import *

import src.utils.countries_map
reload(src.utils.countries_map)
from src.utils.countries_map import *

import src.utils.origin_of_beer
reload(src.utils.origin_of_beer)
from src.utils.origin_of_beer import *

Import Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import numpy as np
import os
from pathlib import Path

from src.utils.utils_functions import *
from src.utils.clustering_GMM import *
from src.utils.aabv_analysis import *
from src.utils.nbr_rating_analysis import *
from src.utils.user_analysis import *
from src.utils.origin_of_user import *
from src.utils.origin_of_beer import *
from src.utils.countries_map import *

# Part 1 Understanding and treating the data

## Part 1.1 Data conversion

The first step in our analysis is to ensure all data is represented consistently across the project. This involves converting the original **.txt** files containing ratings and reviews into **.csv** format. Each file was examined carefully, the strings were stored into dictionaries representing key data fields. More details on the conversion process and methodology can be found in **data/TxtToCsv.ipynb**

The CSV files can be found on the following link: https://drive.google.com/drive/folders/1lcRRxlPpcyAcqJzanlwcyb5Vmip0s7_D?usp=sharing
(You will need to ask for permission to see the files)

## Part 1.2 Data exploration

With the data now in a consistent format, we begin exploring deeper the datasets to understand their links and features. We learn especially that some breweries, beers, users matches between the websites. Moreover, their might be duplicates within the datasets, with some users having multiple accounts. Breweries also have duplicates: i.e a single brewery in ratebeer can correspond to up to 3 breweries in advocate. The goal of this notebook was also to look at the percentage of Nan values for the ratings, and to understand the different variables. Further explanations can be found in **data/data_understanding.ipynb**.

Furthermore an other Jupyter Notebook explores Nan values in the dataset. It is mainly a secondary file that was used to evaluate the percentage of Nan in columns of a dataframe. In there we look at the min/max value of the different grades, where we noticed that both datasets not necessarily contain the same range for their grades. More information can be found in **data/data_cleaning.ipynb**.

Some rows contain NaN values in the datasets. Since certain parts of the analysis do not require every feature, we handle missing data filtering based on the requirements of each analysis section.

## Part 1.3 Dataset merging
To enhance the robustness of the analysis, we merge data from both RateBeer and BeerAdvocate. This approach increases the number of ratings per beer enabling a higher reliability and enhancing the controversiality analysis.

The aim is to have a general dataset for users, beers, breweries and ratings. For users, breweries, ratings each one contains a new id, the old id from both datasets except for ratings. It also contains the name, location and other information. If it is match we often make a choice from which dataset to use the information, for example as two breweries in advocate are a single in ratebeer we decided to take the name of the brewery from ratebeer. We filter out the matched informations as a single one.

For ratings we have every rating of both datasets, even the one matched twice. We decided to keep them both as we noticed that comments may differ in the grading and textual description. We added columns corresponding to the new beer, user and brewery id. Certain attributes where deleted as we assumed that we could recompute them again, or if need could reload the old files. Finally we gave an id to each rating, the dataset of origin as 'rb' or 'ad', and we added a column called matched if filled with a number contains the id of the rating it is matched with.

Further information can be found add **data/merge_into_onedataset.ipynb**. The transformed data can be found at the following link:
https://drive.google.com/drive/folders/1McQ7BU24mEsEqouulOPqrmtQJ47E6ZP8?usp=sharing
(You will need to ask for permission).


**Data Loading**

For the whole the next cell calls the different datasets and is used for the rest of the notebook.

In [ ]:
root = Path(os.getcwd()).parent

# Files are located in Dataset folder
dataset_path = os.path.join(root,'Dataset')

FULL = "full"
FULL_PATH = os.path.join(dataset_path,FULL)

breweries_df = pd.read_csv(os.path.join(FULL_PATH, 'breweries.csv'))
beers_df = pd.read_csv(os.path.join(FULL_PATH, 'beers.csv'))
users_df = pd.read_csv(os.path.join(FULL_PATH, 'users.csv'))
ratings_df = pd.read_csv(os.path.join(FULL_PATH,'ratings.csv'))
sentiment_df = pd.read_csv(os.path.join(FULL_PATH,'beer_reviews_with_sent.csv')) # FAUDRAIT QUON CHANGE LE NOM
sentiment_analysis_df = pd.read_csv(os.path.join(FULL_PATH,'sentiment_analysis.csv'))
ratings_with_sent_df = ratings_df.join(sentiment_df.set_index('id'), on='id')
del sentiment_df

**Grade distribution**

In [ ]:
compute_stastics(ratings_df) # FAUDRAIT MODIFIER CETTE 
# FONCTION POUR QU'ELLE PRINT AVEC DES RETOUR à LA LINE

In [ ]:
plot_var_distrib_violin_grades(ratings_df=ratings_df)

**Grade transformation**
We notice during the data cleaning that grades do not have the same range and resolution on both webistes. 

BeerAdvocate allow the user to rate five attributes (aroma, palate, taste, appearance, overall) on a scale of 1 to 5 with a resolution of 0.25, while RateBeer allow the user to rate aroma and taste between 1 and 10, appearance palate between 1 and 5 and overall between 1 and 20, all with a resolution of 1.

To perform our analysis, we must transform the grades to have the same scale between the two dataset. We decided to set the grade between 1 and 5 as most attributes are graded with this range. This is done by converting RateBeer scales according to the following formula : 

$$ \frac{\text{grade} - \text{min}}{\text{max} - \text{min}} \cdot 4 + 1 $$
- with $\text{max}$ the largest grade one can give to a given attribute.
- with $\text{min}$ the lowest grade one can give to a given attribute.

In [31]:
ratings_df = recompute_grade(ratings_df, min_grade_value = 1, max_grade_value = 5, attributes_of_interest = ['appearance', 'aroma', 'palate', 'taste', 'overall', 'rating'])

**Grade distribution after rescaling**

In [ ]:
plot_var_distrib_violin_grades(ratings_df=ratings_df)

As we can see, it results in attributes having different resolutions. This might be a problem when computing some metrics to define the controversiality and comparing between attributes. This will be discussed further.

# Part 2 Exploring the definition of controversiality

This part aims to determine and label which beers are controversial and which are universal. To do this, we explore different aspects defining the controversiality of a beer. 

What does controversial mean : "subject of intense public argument, disagreement, or disapproval" [1]. As described, this depends on the opinions of the people. As a result, this analysis only depends on the fields the users can fulfill, namely, the different ratings : appearance, aroma, palate, taste, overall, and the textual reviews

We first try to label the beers using the grades provided by the user, later we will try to add the sentiment analysis and observe if it helps in classifying the beers.

The metric used for analyzing controversiality is the **variance**. It measures the spread of the data around the mean, reflecting the level of disagreement among ratings. Furthermore, it is consistent across all attributes as it is dimension-independent, relying only on the grades given by the users.

## Part 2.1 Ratings and reviews filtering

As mentioned earlier, controversiality depends on disagreement in opinions. Beers with few ratings are more likely to show high variability (e.g. two opposing opinions). To ensure reliable insights and meaningful analysis, we exclude beers with fewer number of ratings or reviews than a specified threshold. (((((((((((Later, we might apply a weighting factor based on rating count to further refine the controversiality analysis, according more importance to more rated beers.)))))))))))

Let's try to find a reasonable threshold.



In [ ]:
plot_threshold_filtering(ratings_df)

A choice of threshold cannot be easily done as we need to balance two opposing considerations :
- Maximize the number of data : a lower threhsold includes more beers in the analysis. However, beers with few ratings might introduce noise as calculated variances may not be statistically reliable if based on too few opinions.
- Meaningful variance : a higher threshold filter beers with too few ratings, ensuring the variance is computed on enough samples, hence meaningful and reliable for further analysis.

From the plot, we observe a steep decline in the percentage or remaining ratings when threshold increase from 5 to 50 approximatively. Beyond 50, the decline becomes more flat, meaning less impact of the threshold on remaining ratings.

We also observe a steep decline in the percentage of beers remaining when threshold increase from 5 to 25 approximatively.

To take into account our 2 considerations, we avoid beers below 10 ratings, being unreliable, but choose a threshold below 35 to keep at least 10% of the beers and 75% of the ratings.

The choice is thus : ???????????????????

In [ ]:
beer_ratings, breweries_df_new, beers_df_new, users_df_new = filter_ratings_new(ratings_df, beers_df, breweries_df,users_df, threshold=25, attributes=['appearance', 'aroma', 'palate', 'taste', 'overall'])

ADAPT PERCENTAGE TO TELL IF BEFORE OR AFTER DROPPING NANS !!!!!!!

## Part 2.2 Naive analysis


We first try here a naive analysis of the controversiality, observing if an easy classification of beers can be done. Which attributes do we want to keep to define controversiality ?

Controversiality can be analyzed in different ways. First naive analysis were tried as follow :
- Compute the variance of each attributes (appearance, aroma, palate, taste, overall) for each beer, studying which seems to be the most controversial by looking at the distribution of the variances.
- **Overall** attribute represents the complete opinion of someone on a beer. It might be the most important grading parameter to take into account to define the controversiality of a beer. We can try to compute how variances of other attributes fluctuate as a function of **overall** variance. First, compute the variance of **overall** attribute. Then classify the beers using only the variance of **overall** attribute. Finally compute the variance of the remaining attributes and study their controversiality.

In [33]:
# Compute the variance of the attributes and study controversiality
attributes_variance = compute_variance_per_attribute(beer_ratings, ['appearance', 'aroma', 'palate', 'taste','overall'])

In [ ]:
plot_var_distrib_violin(attributes_variance)

In [ ]:
attributes_variance.describe()

As we can see, it seems that appearance and palate have the highest mean variance. They also have quite different distribution than others. This will be further investigated.

In [ ]:
# Analyzing attributes from using overall variance as classification parameter
# Keeping the 10 most and least % as controversial and universal
[controv_attrib_var, univ_attrib_var] = classify_percentage_distribution(beer_ratings, ['appearance', 'aroma', 'palate', 'taste'], ['overall'], 10)
plot_var_boxplot(controv_attrib_var, univ_attrib_var)


In [ ]:
controv_attrib_var.describe()


In [ ]:
univ_attrib_var.describe()

From this, we can observe the following :

When the beer is in the top 10% highest **overall** variance, **palate** and **taste** are dominating the controversiality, having the highest variance. It seems to indicate that when there is strong disagreement on a beer, stronger reason for that stem from these two attributes. 
When the beer is in the top 10% lowest **overall** variance, **taste** and **aroma** are the least controversial, having the lowest variance. It seems to indicate that when there is universal opinion on a beer, stronger reason comes from these two attributes.


It seems that all attributes variances are positively correlated with **overall** variance. **Taste** seems to be the most correlated one. As a result, most of overall variance might be explained by this attribute.



Let's look at the correlation between the variance of the four attributes with the **overall** attribute.

In [ ]:
# Correlation of the attributes variances with overall variance
correlations = compute_correlation(attributes_variance, ['overall'])
plot_correlation(correlations)

As expected, the variance of the attributes are positively correlated with the variance of **overall**, well defining the global opinion of a user on a beer. On the other hand, a beer might be controversial even having a low variance of **overall** opinion. A beer might have a really high variance on two attributes but not the **overall** one. As a result, we cannot take solely this one for defining controversiality.

After further consideration, we decided that all grading parameter were equally important for our analysis. Controversiality is subjective and a simple decision boundary cannot be made. Some attributes might be correlated together, for example **taste** and **palate**, while others might be completely independent like **appearance**. But most of the time, all are taken into account when a user grades the **overall** parameter. As a result **overall** might be a good starting point to study controversiality, but it does not capture the complex nature of user opinions. It often acts as a weighted average of other attributes, specific to each users, prioritizing different characteristics of the beer.

Controversiality is thus multi-faceted and depend on the interplay of all attributes.

## Part 2.3 Clustering the beers

Through initial analysis, we did not find an easy classification of the beers based on controversiality. Therefore, we decided to use a Gaussian Mixture Model (GMM) approach, incorporating the variances of all five attributes to classify beers as controversial, neutral, or universal. It allows us to account for complex and multidimensional patterns in the data, providing a more meaningful mathematical definition of controversiality.

Let's first perform a GMM analysis from 1 to 10 clusters and observe the negative log-likelihood of the resulting clustering.


In [ ]:
plot_nll(attributes_variance, min = 1, max = 10)

By applying the elbow method on the plot of the negative log likelihood, we see that the optimal number of clusters is 3 (((OUAIS OU 2 MAIS ON PEUT DIRE QUE 3 NOUS ARRANGE ???))). It is ideal since we would like to have three clusters corresponding to controversial, neutral and universal. Now let's define which cluster corresponds to which label.

In [ ]:
labels = visualize_gmm_all_dimensions(attributes_variance, n_components_gmm=3)

In [ ]:
print_label_statistics(labels)

The clusters were computed using variances that were standardized to have a mean of 0 and a standard deviation of 1.

As discussed earlier, we use variance as a key metric to capture controversiality. It is controversial if it has a high variance across its attributes, and universal if it shows low variances. The values displayed on the plot represent the scaled variances. Since all variances are positive, smaller scaled variances (closer to 0) indicate lower variance, while larger values correspond to higher variance. Based on this definition, we define the clusters as follows:

- Cluster 0 : Universal beers, accounting for 46.91% of those with sufficient number of ratings.
- Cluster 1 : Neutral beers, representing 44.04%.
- Cluster 2 : Controversial beers, representing 9.05%.



# Part 3 Extracting information from the textual reviews

In the previous part we used every aspect that the user could grade except the textual reviews. In this part we aim at choosing a model to perform sentiment analysis, and observe if using a score of sentiment analysis can further improve the classification of the beers into controversial/universal.


## Part 3.1: Sentiment Analysis

We will begin by performing a sentiment analysis on the reviews. This will help us gain deeper insights when we later classify the reviews by topics. The primary objective is to identify a reliable, multilingual model. To achieve this, we will compare the performance of various models:  A [BERT base multilingual uncased model](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment), [Google Cloud NLP](https://cloud.google.com/natural-language/docs/analyzing-sentiment?hl=fr), [GPT-4o mini](https://openai.com/index/gpt-4o-mini-advancing-cost-efficient-intelligence/) and a 
[distilbert base multilingual cased model](https://huggingface.co/lxyuan/distilbert-base-multilingual-cased-sentiments-student).

We will compare the performance of each model on the first 6,000 reviews. We chose 6,000 reviews because it is a large enough sample for a minimally biased analysis while helping save on API credits.

Each review was labeled as Positive (1), Neutral (0), or Negative (-1).  We kept the result of this analysis in the [_sentiment_analysis.csv_](https://drive.google.com/file/d/1attiqoXvTSruf6DDXodvU7jFvLKZCOHr/view?usp=sharing) file.

We then compared the models’ performances based on exact similarity and similarity within a range of ±1. The latter is particularly relevant because it allows us to identify models that may produce false positives or false negatives.

In [8]:
[df, exact_similarity_columns, plus_minus_1_similarity_columns] = compute_similarity_scores(sentiment_analysis_df)

We plot the result in the following bar plots for the exact and ±1 similarity scores across different pairs.

In [ ]:
plot_sentiment_similarities(df, exact_similarity_columns, plus_minus_1_similarity_columns)

First, we observe by looking at the bar plots that the BERT and Distlilbert, the Google Cloud NLP and GPT-4o mini models, as well as the BERT anf Google Cloud NLP models show the highest similarity for exact sentiment, exceeding 65%.

Additionally, the Google Cloud NLP and GPT-4o mini models have nearly 100% similarity when considering a margin, followed closely by the BERT-based model and GPT-4o mini, and then by the DistilBERT model and GPT-4o mini.

Given that GPT-4o mini, the BERT model and Google Cloud NLP are very similar in the second graph, that GPT-4o mini is falling behind in the exact sentiment comparison and the DistilBERT model is behind the other models in both comparisons, we should use either Google Cloud NLP or the BERT-based model. Even though Google Cloud NLP has a slightly better similarity in both comparisons, it also an expensive api for large datasets. We will therefore use the BERT-based model for the final sentiment analysis.

## Part 3.2: Comparing the distribution with the other attributes

Now that we have a sentiment score for the comments with a textual review we can compare the distribution with the other grades. We want to see if it adds information, or if it is related with another attribute. For this we first need to recompute the grades provided by the sentiment analysis as the score is between -1 and 1. We can then filter the remaining beers and perform our statistical analysis.

In [ ]:
#Recomputes the grades
ratings_with_sent_df = recompute_grade(ratings_with_sent_df, min_grade_value = 1, max_grade_value = 5, attributes_of_interest = ['appearance', 'aroma', 'palate', 'taste', 'overall', 'rating','sentiment_bert'])

#Filters the remaining beers, breweries, users and ratings
beer_ratings_new_sent,breweries_df_new_sent, beers_df_new_sent, users_df_new_sent = filter_ratings_new(ratings_with_sent_df, beers_df, breweries_df,users_df, threshold=25, attributes=['appearance', 'aroma', 'palate', 'taste', 'overall','sentiment_bert'])

#Computes the variance of the various attributes of interest
attributes_variance_sent = compute_variance_per_attribute(beer_ratings_new_sent, ['appearance', 'aroma', 'palate', 'taste','overall','sentiment_bert'])

We note that by keeping only comment with a textual description, we keep 19.22% of the initial beers. However we still keep 56.10% of the comments.

As mentioned previously we want to test if the distribution of the variance provided by the sentiment analysis is similar to the distribution of another attribute. Therefore, we perform a T-Test statistic to evaluate this hypothesis. The T-Test is applied on the variance of each attribute. The H0 hypothesis is the following : the true mean of the variance of a given attribute between the different beers are equal. H1 is the following : the means are different.
We decided to plot the results in a Heatmap where the value is the p-value of the statistical test between the variance of two attributes. The diagonal should be 1 as it is twice the same distribution.

In [ ]:
t_test_statistic(attributes_variance_sent, attributes_of_interest = ['appearance', 'aroma', 'palate', 'taste','overall','sentiment_bert'])

From the previous plot we can reject the H0 every time with a significance level of 5%. This means that the sentiment analysis provide additional information to the attributes we already have. 

Now, we want to see how variance of the sentiment score is correlated with the other attributes.

In [ ]:
correlations = compute_correlation(attributes_variance_sent, ['sentiment_bert'])
plot_correlation(correlations, 'Correlation of attributes variances with the variance of the sentiment analysis')

From the above plot we see that the sentiment score variance is positively correlated with all the other attributes. Every correlation is around 0.4, however the appearence is particularly low at around 0.31. This could tell us that the textual review does not provide as much information about the variance of the appearance than for other attributes. However such hypothesis would need to be studied more carefully with keywords analysis.

## Part 3.3: Clustering and comparison against the previous cluster

In Part 2, we clustered the beers using GMM on the attributes. Now we can add the information provided by the sentiment analysis and compare the different clusters. We aim at evaluating the clusters choosing how we will define controversiality for the rest of our analysis. Now let us first perform the GMM clustering. For meaningful comparison, we use three clusters as done in Part 2.

In [ ]:
labels_sent = visualize_gmm_all_dimensions(attributes_variance_sent, n_components_gmm=3)

In [ ]:
print_label_statistics(labels_sent)

Similarly to the previous part, we observe what the different clusters look like across the various attributes.
The clusters were computed using variances that were standardized to have a mean of 0 and a standard deviation of 1.

Using again our definition of controversiality, a beer is controversial if it has a high variance across its attributes, and universal if it shows low variances. As before, the values displayed on the plot represent the scaled variances. Since all variances are positive, smaller scaled variances (closer to 0) indicate lower variance, while larger values correspond to higher variance. Based on this definition, we define the clusters when adding sentiment analysis as follows :
- Cluster 0 : Neutral beers, accounting for 36.74%
- Cluster 1 : Universal beers, representing 55.66%.
- Cluster 2 : Controversial beers, representing 7.60%. 

Now we want to compare the two clustering made with and without sentiment analysis and see if they differ. This will be done only on the beers containing textual reviews.



In [ ]:
compare_two_clustering(attributes_variance, attributes_variance_sent, labels, labels_sent, [1,2,0], [2,0,1])


We see that clusters do not match in 79% of the cases. We believe the cause is coming from the sentiment score. During the evaluation process of the model we saw that we had 66% of perfect accuracy if the texts are classified as 0,-1 or 1. Therefore it introduces a higher variance since it is noisy. We also see that number of beers per label differs from the initial clustering.

## Part 3.4: Understanding the difference between the two clusters

We previously found two clustering, one with the sentiment analysis and one without. The clusters labeled 79.6% of the beers in the same way. However we want to understand what is the reason. To have a faire comparison, we want to create a new clustering using only the ratings having a textual review, but without using it. This is done to mitigate the source of difference. Between the two previous clusters we had two variables which changed:
- The number of ratings
- The sentiment score

By clustering without the sentiment score on the ratings having only a textual review, we can compare the obtained clusters with the preivously found clusters, and thus explain the difference.

In [ ]:
labels_sent_without_sent = visualize_gmm_all_dimensions(attributes_variance_sent[['appearance','aroma','palate','taste','overall']], n_components_gmm=3)

In [ ]:
print_label_statistics(labels_sent_without_sent)

By looking at the plot we can define by using the same methodology as described before the following clusters:
- Cluster 0 : Neutral beers, accounting for 36.39%
- Cluster 1 : Universal beers, representing 56.12%.
- Cluster 2 : Controversial beers, representing 7.49%. 

We first compare the two clusters obtained on the ratings having a textual review.

In [ ]:
compare_two_clustering(attributes_variance_sent, attributes_variance_sent, labels_sent_without_sent, labels_sent, [2,0,1], [2,0,1])


We have a 95% correspondance between the labels. The sentiment score would only change the label of 5% of the beers.
Now let's compare the two clusters generated without the sentiment score.

In [ ]:
compare_two_clustering(attributes_variance, attributes_variance_sent, labels, labels_sent_without_sent, [1,2,0], [2,0,1])


We have only an 80% of correspondence. This means that the difference of the number of beers used for the clustering results in a 20% difference.
This is bigger than the difference due to the sentiment score.

Our initial explanation from Part 3.3 is contradicted here. The sentiment score generates a difference of 5% in labeling, which is not an issue. However, the number of beer changes drastically, therefore we prefer to keep the initial clusters. Furthermore since we have only 5% of difference with or without sentiment, it does not provide sufficient supplementary information to balance out the loss of generalization du to the decrease of the size of the dataset. Furthermore those 5% could be due to the accuracy of 60% of the model. In the end we would need real labels to have a sentiment score which would serve as ground truth and on which we could rely. For all those reasons we decided to pursue the analysis of the next parts with the clusters discovered in Part 2.

In conclusion, sentiment analysis provides another attribute which adds information compared to the other attributes. However we need more accurate results or at least results on which we could rely more confidently to use them for the next step of this analysis. Lastly, using only ratings with textual review would drastically shrink the size of the dataset.



# Part 4 Analyzing potential reasons of controversiality

After having labelled the various beers into three categories, we study some underlying factors behind the variance of the grading of some beers. We analyze the effect of the alcohol by volume, the number of ratings of the beer, the level of expertise of the user and the origin of the user/beer. By level of expertise, we mean that we want to separate them into three categories, which we will delve into later. 

## Part 4.1 Effect of the alcohol by volume

An attribute given for the beers is the alcohol by volume (abv) level of the beers. We want to test the correlation between the abv and the frequency of a beer being labeled as controversial. 

First let us look at the distribution of beers with a certain amount of abv, and the proportion of each each class corresponding to an abv level. 

In [ ]:
range_grouped_abv = plot_distribution_rating_abv(attributes_variance, beers_df_new[['id','abv']], labels, save = False, interactive = False, label_list=[1,2,0])

# 2ND PLOT IS WRONG CORRECT IT WARNING IMAGE (I plotted the none interactive since the interactive values are wrong)

From the plot we notice that most of the beers have an abv between 0 and 15. Furthermore, for beers upper 22 of abv, since the beers having enough ratings to be taken into the analysis become less frequent, we notice the proportions oscillate much more. 

We observe that at high abv, the beers are more frequently classified as controversial. However the results needs to be taken with a grain of salt, since we have very few beers at this level, providing not reliable enough results.

We want now to compute the correlation between the abv level and the proportion of the classified beers and the total number of beers. This will be done for beers with an abv smaller than 22, as we believe the oscillation could falsify our results.

In [ ]:
correlations = compute_correlation(range_grouped_abv[range_grouped_abv.avg_abv<20].drop(columns='abv_range'), ['avg_abv'])
plot_correlation(correlations, title = 'Correlation of the alcohol by volume with the number and proportion of each class')

From the above plot we observe without any surprise that the number of beers in total and for each class is negatively correlated with the abv. However more suprisingly the abv is positively correlated with the average frequency of the universal beers. This can lead us to believe that of to 20, if the abv increases, then opinions are more universal (could be all positives or negatives).

# Do we do the analysis for the beers with abv >20 (or whole) ? JE DIRAS PLUÔT QUE LES BIèRES EN DESSOUS DE 20 NON ?

## Part 4.2 Effect of the number of ratings

Another attribute we have is the number of ratings for a beer. We want to perform the same analysis as for the abv. First, let us look at the distribution of the number and proportion of beers classified within one of the three categories for a certain amount of number of ratings.


In [ ]:
grouped_df_nbr_rating = plot_distribution_rating_nbr(attributes_variance, beers_df_new[['id','nbr_ratings']], labels, label_list=[1,2,0])

Similar to the the analysis of the abv, we note a point where the amount of beers starts to be small and creates oscillation. This point is at around 5'000 ratings. However in this case this happens at the final possible amount of ratings. At around 2'000 ratings we can already notice small fluctuations, but the trends are similar to the previous lines. 

Now let us look at the correlation between the number of ratings and the total number OF WHAT ?????? and frequency. 

In [ ]:
correlations1 = compute_correlation(grouped_df_nbr_rating.drop(columns='nbr_ratings_range'), ['nbr_ratings'])
plot_correlation(correlations1, title = 'Correlation of the number of ratings level with the number and proportion of each class')

This time we plot the correlation for every number of rating, unlike the abv where we had a threshold for the abv. The absolute correlation values are smaller than on the abv graph. Furthermore we notice two positive case: the universal and controversial controversy. However this remains very small. Also without any surprise the total_number has negative correlation which decreases in the three cases.

## Part 4.3 Effect of the expertise level of the user
Another attribute we believe could influence the controversiality of a beer is the expertise level of the user. By expertise level we mean that if a user made 5 ratings while another made 5'000, they might neither grade the beer in the same way (e.g. being more severe), nor attribute the same importance to different characteristics of the beer. As a result, we believe that some category might be more responsible of the controversiality of the beer than another. We will compute the proportion of beer labeled as controverisal/universal/neutral for each category of user and analyze the results.

First we need to find a threshold to classify the users. We want to create three classes: connoisseur, enthusiast and novice. Note that this is not based on the actual numbers of beers an individual has tested but the number of beers he has **rated**.  (DEPENDS ON THE THRESHOLD DECIDED PREVIOUSLY).

Note that the number of ratings of a user is the number of ratings he made after dropping the ones for which the beer has too few ratings  (ARE WE SURE ?)

In [ ]:
plot_histogram_nbr_ratings_total(users_df_new)

By inspecting the previous graph, we can set two thresholds. One at around 20 ratings and the second around 800 ratings. The first threshold corresponds to the points where the first line changes slope. After this point the frequency starts to oscillate. The second points corresponds to the point where the oscillations increases drastically. Furthermore with the printed stats we can see that 50% of the users made 3 or less ratings. In the end our first threshold is after 75% quantile.

If we sum up the classes we have:
- Novice: <=20 ratings, it contains 75.6% of the users
- Enthusiast: >20 ratings and <=800 ratings, it contains 21.3% of the users
- Connoisseur: >800 ratings it contains 2.6% of the users

We can now compare the proportion of the beers having different labels between the different level of users.

In [ ]:
user_df = classify_user_rating_level(users_df_new, enthusiasts_level=20, connoisseur_level=800)
plot_proportion_controversial_per_category(ratings_df = beer_ratings, users_df = user_df, beers_df = beers_df_new, label = labels, label_list = [1,2,0])

From the graph we notice no clear difference in the category of the user.
We can still highlight the following :
- A novice user is twice as likely to have controversial opinions compared to the other two groups (~10% vs ~5%). Consequently, novice users appear less frequently in the universal category (~55% vs ~60%).
- Enthusiasts and connoisseur have nearly identical proportions across all three labels. We can still notice than enthusiasts tend to have a bit more controversial opinions than connoisseurs.

We could conclude that en gros plus t'as fait d'avis plus t'as de proba d'avoir potentiellement l'avis "juste" d'une bière (logic) puisque t'es moins controversial et donc tu sais mettre tes préférences de côtés pour bien noter une bière un truc comme ça

# Part 5 Countries influence on the controversiality

Lastly for our project, we wanted to analyze the influence of the origin of the user/beer. The idea initially came from an other project proposal, but we felt it could be integrated into this project. Our main motivation, is that we know that certain countries have rivalities with others, or certain countries forbid alcohol. Therefore they may grade differently the beers. 

## Part 5.1 Importance of the origin of the user

In this part we want to look at the influence of the origin of the user. For this we plan on understanding if a certain countries of origin provide comments to more controversial or universal beers. First we want to have a glimpse at the data and understand from where do the users come. 

# JE SAIS PAS SI ON LE PECISE OU PAS MAI TOUTE LES ANALYSES SONT FAITES PAR RAPPORT AU DATASET RÉDUIT. (EN TERME DE RATING/Brewey/User/beer)

In [ ]:
plot_distribution_number_ratings_per_country_of_origin_of_user(users_df_new, beer_ratings, N = 80)

We see from the plot that countries such as the US, provide much more comments about ratings. Compared to certain countries it is 10^4 or even more. If you change the parameter N of the function, you can look at other countries. We also note that 10^5 ratings come from a user with an unknown location.

In [ ]:
users_df_corrected = plot_frequency_user_from_country_label(beers_df_new,users_df_new, beer_ratings, labels, label_to_match = 0)
users_df_corrected = plot_frequency_user_from_country_label(beers_df_new,users_df_new, beer_ratings, labels, label_to_match = 1)
users_df_corrected = plot_frequency_user_from_country_label(beers_df_new,users_df_new, beer_ratings, labels, label_to_match = 2)

We note from the previous plots that users from small countries tend to have a higher frequency to participate in controversial beer. In this case Mongolia seems to be the country with the highest proportion. The universal frequency is approximataly the same for a lot of country, particularly the countries in Europe and America seem to have a higher proportion for universal ratings.

## Part 5.2 Importance of the origin of the beer

Now that we analyzed the country of origin of the user, we can also compute the frequency of a labelled beer for the location of the beer.

In [ ]:
plot_distribution_number_ratings_per_country_of_origin_of_beer(beers_df_new, breweries_df_new, beer_ratings, N = 100)

# MAIS SUIT DIT QUE ON POUVAIT PEUT ETRE DIVISÉ OU SOUSTRAIRE AUSSI LE SCORE< PAR LA PROBA QUE UNE BIÈRE soit CONTROVERSIAL?NEUTRAL/UNiversal. GENRE ÇA PERMET DE MIEUX VOIR SI UN PAYS EST AU DESSUS OU EN DESSOUS.


In [ ]:
compute_proportion_label_per_country(breweries_df_new,beers_df_new, attributes_variance, labels, 0)
compute_proportion_label_per_country(breweries_df_new,beers_df_new, attributes_variance, labels, 1)
compute_proportion_label_per_country(breweries_df_new,beers_df_new, attributes_variance, labels, 2)

As a reminder we had from part 2.3 the following result: 46%, 44% and 9% of the beers are respectively universal, neutral and controversial. We would expect the countries to be around this percentage, even more the countries with more beers such as the United States of America. 

From the plot we see that the European countries tend to be around those values. The United States however seems to have a higher percentage of universal beers. Furthermore the smaller countries, south american, asian and some african countries tend to have a higher proportion of controversial beers.

Also since some countries have fewer beers it is more easier for them to have a higher proportion of controversial beers for example. So the more beers does the country have beers, the more should the country reach the proportions mentionned above.

We can imagine that due to the large amount of beers, that the Americans are responsible for making a beer more universally liked, but since other countries have a smaller percentage of the total amount of beers, they would tend more to controversial beers. J'AI IMPROVISÉ UNE EXPLICATION

## Part 5.3 Relation between origin of the user and origin of the beer

Now that we have analyzed the frequency for the country of origin of the user and of the beer, we want to analyse the relation between the both. We want to see if certain countries tend to participate more easily to make a beer controversial for beers of their country or beers of another country. For this we propose a metric to analyze this for a particular class.

Let $s_x^{class}$ be the frequency of a certain class for beers of the country similar as the country of origin of the user, and let $s_y^{class}$ be the frequency for a certain class for beers of the country different to the country of origin of the user, we obtain that the score is defined as:

$$Score^{class} = s_x^{class} - s_y^{class}$$

Now let is use an example. Imagine we compute the frequency of controversiality. If the country tends to classify with a higher frequency as controversiality for beers from the same country than beers from a different country we get that the score tends to +1. If the score goes to -1 it means that the country has a higher frequency of beers labelled as controversial from foreign countries than other countries.

In [ ]:
plots_statistics_origin_beer_wrt_user_origin_sum(beers_df_new, breweries_df_new, users_df_new, beer_ratings, labels, label_to_match = 0)
plots_statistics_origin_beer_wrt_user_origin_sum(beers_df_new, breweries_df_new, users_df_new, beer_ratings, labels, label_to_match = 1)
plots_statistics_origin_beer_wrt_user_origin_sum(beers_df_new, breweries_df_new, users_df_new, beer_ratings, labels, label_to_match = 2)

From the above plot we note interesting results:
- South America and Asia notably (other countries too) tend to grade more universally beers from foreign countries.
- South America again is controversial for the beer from their countries. 
- The United States of America and Kazachstan are in an opposition trends to the other countries: beers from their country are more universal, and they rate beers as controversial more for beers of foreign countries.
- We could have expected the results to be around 0, as we could expect the users to grade beers with no preference of certain countries. This is the case for some Asian and African countries.
- Finally by looking at the statistics of the score we see that the mean is positive for controversiality and neutrality (14% and 13%), the mean case is that countries are more controversial and neutral for beers of their countries. Furthermore the score is in mean negative for universality(27%) (twice as much as the other two.), which could lead us to believe that country grade more universally beers from foreign countries.

Note that the mean is not weighted with the amount of beers of a country, each country has the same influence on the mean score.

Also since the US produces the most beers, and has the most users, it could explain the fact that it is against the trend as they would grade a beer as universally, which other users would too but they would be considered as foreign countries but smae country for the US.



## Part 6 Future work

Future work could include the following :
- In part 2.1 and ???, when filtering beers to ensure a robust and meaningful variance for further analysis, futher work could include lowering the threshold and weight the beers according to their number of ratings (COMMENTARY TO DEVELOP)
- KEYWORD ANALYSIS

# Part 7 Conclusion

As a conclusion, through this notebook we have seen that it is difficult to classify the beers into universal and controversial classes. Therefore we switched to GMM. We ended up with a ratio of 46/44/9 for universal, neutral and controversial respectively. Furthermore the sentiment analysis provides further information about the beers. If we cluster on it, the labels remain at 95% similar as if we do not cluster with the sentiment analysis. We could use the sentiment analysis, however due to the 60% +1/-1 accuracy we decided that it may influence the result in a way we do not want to. 

Following this we can analyze the relation with the alcohol by volume, number of rating and the user level. We need to be careful of edge cases. The universal frequency is positively correlated with the alcohol by volume and the number of ratings. The later one is also positively correlated with the controversial frequency. The abv has a higher absolute correlation value with the frequencies, it may be more of an underlying factor than the number of ratings for the classification. Finally we noticed that novice rater, representing a huge part of the users are more prone to rate non universal beers than the other graders.

Finally the nationality plays a certain role, for the origin of the beer but also the origin of the user. We have seen that particularly the United States, which produces the biggest amount of beers (of the dataset) and has the biggest amount of users, has a trend often different from the other countries to grade a particular beer. 

This work contains a lot of nuancing and doors that where not followed due to a lack of time. Therefore a lot of work could be done to pursue this study.

Thank you for reading this notebook, enjoy your holidays. 



In [ ]:
import torch

visible_joints = torch.tensor([[1,0,0,2,0,1,0,0,0,1,1,1,1]])
print(visible_joints)
visible_joints.nonzero(as_tuple=True)[1]